In [ ]:
def prepare_custom_dict(input_data, settings):
    # Default settings for API request
    default_dict = {
        "model": "gpt-4",
        "temperature": 1.05,
        "max_tokens": 500,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "messages": []
    }
    
    # Initialize messages list and dict_to_send
    messages = []
    dict_to_send = default_dict.copy()

    # Update system message if present in settings
    if settings and "system" in settings:
        messages.append({"role": "system", "content": settings["system"]})

    # Filter settings to only include keys that are in default_dict
    if settings:
        filtered_settings = {k: settings[k] for k in settings if k in default_dict}
        dict_to_send.update(filtered_settings)

    # Add user message to messages list
    messages.append({"role": "user", "content": input_data})

    # Update dict_to_send with messages
    dict_to_send["messages"] = messages

    return dict_to_send



def get_chat_response(input, settings):
    # if settings dict does not contain key "do nothing"
    dict_to_send = prepare_custom_dict(input, settings)
    print("dict_to_send: ")
    print (dict_to_send)

    import funcs_open_ai_chat
    gpt_resp = funcs_open_ai_chat.send_request_to_openai(dict_to_send)
    return gpt_resp


In [ ]:
import funcs_airtable
import funcs_flask
import funcs_open_ai_chat
import funcs_utility



def process_airtable_gpt_response(record_id):
    record_data = funcs_airtable.Get_Record_Data_From_ID("Actions", record_id)
    record_content = record_data.get("Content")
    # print(record_content)

    # settings 
    record_settings =  record_data.get("Settings")
    if record_settings == None or record_settings == "":
        record_settings = None

    else:
        record_settings = funcs_utility.clean_up_variables(record_settings)
    print("settings:")
    print(record_settings)

    # variables in the content
    unique_values = funcs_utility.find_unique_bracket_values(record_content)
    variables_content_dict = funcs_airtable.get_bracket_values_content_dict(unique_values)
    combined_content = funcs_utility.render_template_with_dict(record_content, variables_content_dict)
    print(combined_content)


    full_gpt_response = get_chat_response(combined_content, settings=record_settings)
    print(full_gpt_response)

    
    # gpt_response = full_gpt_response['choices'][0]['message']['content']


    # airtable_response =  funcs_airtable.Update_Record("Actions", record_id, "Response", gpt_response)    
    # current_time = funcs_utility.current_time_london()
    # response_formatted =  current_time + " :\n" + gpt_response + "\n===\n"
    # airtable_response = funcs_airtable.append_data_to_column("Actions", record_id, "Response Log", response_formatted)

    # return full_gpt_response


record_id = "recJ3q7NEWUDAHg8J"
process_airtable_gpt_response(record_id)





In [ ]:
import requests

API_URL = "https://flowise-85mv.onrender.com/api/v1/prediction/8256c1c4-408a-4edc-a40a-1056dd64133c"

def query(payload):
    response = requests.post(API_URL, json=payload)
    return response.json()
    
output = query({
    "question": "What are the main concepts?",
})

print(output)

In [10]:
from dotenv import load_dotenv
import os
import asyncio
from supabase import create_client, Client

load_dotenv()  # Load environment variables from .env file

# Initialize Supabase
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


def update_data(table: str, id: int, new_data: dict):
    """Update data in a table by ID."""
    return supabase.table(table).update(new_data).eq("id", id).execute()



payload = {
    "table": "video_creator_projects",
    "id": 3,
    "new_data": {
        "scenes_list": [
            {
                "position": 0,
                "scene_id": "scene_1"
            },
            {
                "position": 1,
                "scene_id": "morty"
            },
            {
                "position": 2,
                "scene_id": "rick"
            },
            {
                "position": 3,
                "scene_id": "jr"
            },
            {
                "position": 4,
                "scene_id": "scene_4"
            },
            {
                "position": 5,
                "scene_id": "scene_5"
            }
        ]
    }
}

response = update_data(payload["table"], payload["id"], payload["new_data"])
print(response)

2023-09-05 00:55:32,982:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?id=eq.3 "HTTP/1.1 200 OK"


data=[{'id': 3, 'created_at': '2023-09-04T13:02:05.90948+00:00', 'scenes_list': [{'position': 0, 'scene_id': 'scene_1'}, {'position': 1, 'scene_id': 'morty'}, {'position': 2, 'scene_id': 'rick'}, {'position': 3, 'scene_id': 'jr'}, {'position': 4, 'scene_id': 'scene_4'}, {'position': 5, 'scene_id': 'scene_5'}], 'project_name': 'Indonesia'}] count=None


In [9]:
import requests
import json

# Define the payload (your data to be sent)
payload = {
    "table": "video_creator_projects",
    "id": 3,
    "new_data": {
        "scenes_list": [
            {"position": 0, "scene_id": "scene_1"},
            {"position": 1, "scene_id": "morty"},
            {"position": 2, "scene_id": "rick"},
            {"position": 3, "scene_id": "jr"},
            {"position": 4, "scene_id": "scene_4"},
            {"position": 5, "scene_id": "scene_5"}
        ]
    }
}

# URL for the POST request
url = "http://127.0.0.1:5000/supabase/test"  # Replace with your actual URL

# Optional: headers
headers = {
    'Content-Type': 'application/json'
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)

# Display the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())


Status Code: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)